## Sign language classifier

Importing Libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import multilabel_confusion_matrix
import cv2
import time

In [ ]:
print(cv2.__version__)

### Preparing Dataset

In [ ]:
def preprocessing(train_path, test_path):
    
    df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
        
    df_numpy = np.asanyarray(df)

    train_X = df_numpy[:, 1:]
    train_Y = df_numpy[:, 0]

    train_X = train_X.reshape(27455, 28, 28, 1)
    train_Y = train_Y.reshape(27455, 1)

    plt.imshow(train_X[45].reshape(28, 28))
    plt.show()

    df_test_numpy = np.asanyarray(test_df)

    test_X = df_test_numpy[:, 1:]
    test_Y = df_test_numpy[:, 0]

    test_X = test_X.reshape(7172, 28, 28, 1)
    test_Y = test_Y.reshape(7172, 1)

    test_X = test_X/1.0

    plt.imshow(test_X[909].reshape(28, 28))
    plt.show()
    
    
    return train_X, test_X, train_Y, test_Y
    
    
train_X, test_X, train_Y, test_Y = preprocessing("D:\Datasets\Sign Language Mnist\sign_mnist_train.csv", "D:\Datasets\Sign Language Mnist\sign_mnist_test.csv")

In [ ]:
def create_model(path=None):
    if(path==None):
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(75, (3, 3), strides=1, padding="same", activation=tf.nn.relu, input_shape=(28, 28, 1)),
            tf.keras.layers.MaxPool2D((2, 2), strides=2, padding="same"),
            tf.keras.layers.Conv2D(50, (3, 3), strides=1, padding="same", activation=tf.nn.relu),
            tf.keras.layers.MaxPool2D((2, 2), strides=2, padding="same"),
            tf.keras.layers.Conv2D(25, (3, 3), strides=1, padding="same", activation=tf.nn.relu),
            tf.keras.layers.MaxPool2D((2, 2), strides=2, padding="same"),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation=tf.nn.relu),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(25, activation=tf.nn.softmax)
        ])
        
    else:
        model = tf.keras.models.load_model(path)
        
    return model

model = create_model()

### Compiling Model

In [ ]:
def compile_model(model):
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    
compile_model(model)

### Summary of Model

In [ ]:
model.summary()

### Fiting model

In [ ]:
def fit_model(model, train_X, test_X, train_Y, test_Y):
    
        
    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('accuracy')>0.99):
                print("\nReached 99% accuracy so cancelling training!")
                self.model.stop_training = True


    callme = myCallback()


    train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=(1/255.0),
                                                                height_shift_range=0.2,
                                                                width_shift_range=0.2,
                                                                zoom_range=0.2,
                                                                rotation_range=0.2,
                                                                shear_range=0.2,
                                                                horizontal_flip=True)
    validation_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=(1/255.0))
    
    
    history = model.fit(train_gen.flow(train_X, train_Y, batch_size=125),
                        epochs=20,
                        steps_per_epoch=220,
                        validation_data=validation_gen.flow(test_X, test_Y),
                        callbacks=[callme])
    return history
    
history = fit_model(model, train_X, test_X, train_Y, test_Y)

### Plotting

In [ ]:
model.save("classifier.h5")
def plotty(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc)
    plt.plot(epochs, val_acc)
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.savefig("acc.png", dpi=350)
    plt.show()

    plt.plot(epochs, loss)
    plt.plot(epochs, val_loss)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.savefig("loss.png", dpi=350)
    plt.show()

    img = mpimg.imread('img.png')
    plt.imshow(img)
    plt.show()
    
plotty(history)

### Capture Image

In [ ]:
def capture():
    print("Capturing photo SMILE......")
    time.sleep(5)
    cam = cv2.VideoCapture(0)
    cam.set(3, 28)
    cam.set(4, 28)
    check, frame = cam.read()
    cv2.imwrite("img.png", frame)
    cam.release()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    plt.imshow(gray)
    cv2.destroyAllWindows()
    
capture()

### Predictions

In [ ]:
pred_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=(1/255.0))
c = cv2.imread("img.png",cv2.IMREAD_GRAYSCALE)
new_c = cv2.resize(c,(28, 28))
m = new_c.shape[0]
n = new_c.shape[1]
new_c_r = new_c.reshape(1, m,n,1)
pred_img = pred_gen.flow(new_c_r)
prediction = model.predict(pred_img)
print("predicting on image....")


In [ ]:
plt.imshow(new_c)
plt.show()